In [1]:
# import builtins

# # Save the original print function
# original_print = print

# # Override the print function
# def custom_print(*args, **kwargs):
#     new_args = []
#     for arg in args:
#         if isinstance(arg, float):
#             # Customize the format here
#             new_args.append(f'{arg:.10f}')
#         else:
#             new_args.append(arg)
#     original_print(*new_args, **kwargs)

# # Override the built-in print with the custom one
# builtins.print = custom_print

In [2]:
import pandas as pd
import os
import gzip
import pickle
import openai
import torch.nn.functional as F
import torch
import re
import copy
from tqdm import tqdm
from openai import OpenAI
client = OpenAI()

In [3]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
# enc.encode("<|endoftext|>",allowed_special={'<|endoftext|>'})
enc.encode(" #")

[674]

In [4]:
def get_openai_response(prompt):
    tokens = 4000
    model="gpt-4-turbo-preview"
    # model="gpt-3.5-turbo-0125"
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": """
The current date is Tuesday, September 10th 2023, 10:30am
You are a helpful assistant.
""".strip()
            },
            {
                "role": "user",
                "content": prompt
            }
            ],
        temperature=0,
        max_tokens=tokens,
        top_p=1,
        logit_bias={"674": -1.5},
    )
    choice = response.choices[0]

    text = choice.message.content


    return text

In [5]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware

# add row index
hardware_instances = hardware_instances.reset_index(drop=True)
hardware_instances["row_index"] = hardware_instances.index


In [6]:
unique_values = hardware_instances["hardware"].value_counts().index.tolist()
unique_values.sort()

for a in unique_values:
    print(a)

" nano ledger wallet
"+&\" screw terminals
"2xgpu+2xicarus"
"45 chip boards"
"cgminer --api-listen"
"lip" formed by the different widths of the flat bar
#1 unit
#11 and #13
#12 2-conductor with ground wire cable
#12 wire
#2
#2 pencil
#5-40 (1/2 or 3/4" shcs
#6/#8(linked 15+15a)
$12 grand in black arrow
$16 holmes personal fans
$2 usb hub
$20 plastic box fan
$200 laptops
$25 fans
$4000 rig
$43 upgrade shares
$50 hp pdu
$500 pc
$7 cheap 10 port powered usb wrack
$70= 10-15gh/s
$78 thermaltake sp-750p psus
$7k resistor
%insert%
'13 units
'14 units
'asics'
'shorty' rack for a single rig
(gpu)
(ground) --- (10k-resistor) -------|------- (thermistor) ---- (+5v)
(inno t2 turbo 2s)
(sapphire) 7850
(windows or osx boxes)
*team - t-force / night hawk 16gb (2 x 8gb) ddr4-3200 memory
+ track
+12v fan header
-
--disable-gpu --s all
.333 ghs sticks
.336 asicminers
.4 erupter coupons
.bat file
.bat files
.cmd files
.net
.net 4.0
.net framework
.net software
.py
/dev/ttyacm0
/dev/ttyusb* devices
/dev/

In [7]:
possible_hardware = pd.read_csv('../../hardwarelist/hardware_merged.csv')["hardware_name"].sort_values().tolist()
possible_hardware = [x.lower() for x in possible_hardware]

In [8]:
possible_hardware[:3]

['2x opteron 6128', '2x opteron 6172', '320m']

In [10]:

chunk_length = 50
num_chunks = len(unique_values) // chunk_length + (1 if len(unique_values) % chunk_length != 0 else 0)

hardware_lines = "\n".join(possible_hardware)

# Iterate through the lines chunk_length at a time
# for i in tqdm(range(num_chunks)):
for i in range(num_chunks):
    
    # Calculate start and end indices for slicing
    start_index = i * chunk_length
    end_index = start_index + chunk_length
    chunk = unique_values[start_index:end_index]
    text = "\n".join(chunk).replace("  "," ")

    if not text:  # Make sure the chunk is not empty
        continue
    # check if the file already exists	
    if os.path.exists(f"mappings/hardware_mapping_{i}.txt"):
        continue

    print("processing chunk",i)

    prompt = f"""
User:

Here is a list of hardware that is used for mining bitcoin:

{hardware_lines}



You will be given a collection of strings. Some of them correspond to entries in the list above, while others are unrelated.
Your task is to create a python dictionary that maps the strings from the collection to the names in the list above.
If a string is unclear or not in the table of mining hardware, write "unknown" for the hardware_index value. The names don't need to match exactly.

Example input:
```
sapphire 7950 vapor x
1th/s s2 antminer
1 th mining system
bpmc "blue fury" 2.7 gh/s usb miner v1
18 btc garden v2
antminer s5
block erupter asics
14th s9 of nov.2017 batch
```

Example output:
```python
mapping = {"{"}
    "sapphire 7950 vapor x": "7950",
    "1th/s s2 antminer": "antminer s2",
    "1 th mining system": "unknown",
    "bpmc \\"blue fury\\" 2.7 gh/s usb miner v1": "blue fury",
    "btc garden am-v1", # Assuming a typo or version difference, closest match.
    "antminer s5": "antminer s5",
    "block erupter asics": "unknown",
    "14th s9 of nov.2017 batch": "antminer s9",
{"}"}
```

It is very important that the mapping's inputs correspond exactly to the strings in the collection, including any typo that might be present in the string.
If none of the strings match any hardware names, then write "unknown" for all of them.
If a string can be mapped to a hardware name but the mapping is questionable, write a comment on the same line.

Write the mapping for this collection of strings:
{text}



Assistant:
Sure! Here is the mapping for the collection of strings:
    """.strip()



    print(prompt)
    print("\n"*10)

    response = get_openai_response(prompt)


    print(response)
    print("\n"*10)

    # save to file
    with open(f"mappings/hardware_mapping_{i}.txt", "w") as file:
        file.write(response)

    break






processing chunk 0
User:

Here is a list of hardware that is used for mining bitcoin:

2x opteron 6128
2x opteron 6172
320m
3410
4350
4550
4570m
4650
4670
4730
4770
4830
4850
4860
4870
4890
4x opteron 6174
5450
5550
5570
5650
5670
5750
5750 vapor-x
5770
5830
5850
5870
5970
6310m
6450
6490m
6570
6670
6750
6790
6850
6870
6950
6970
6990
7750
7770
7850
7870 xt
7970
8200 mgpu
8400 gs
8500gt
8600gt
8800 gts
8800 gtx
8800gt
8800gts
9300/nforce 730i
9300ge
9300gs
9400gt
9400m
9500gt
9500m gs
9600gso
9600gso512
9600gt
9600gt zotac
9600m gs
9800gt
9800gt ee
9800gtx
9800gtx+
9800gx2
a10-5800k
a4-3400
a8-3850
a8-3870k
aisen a1 pro
allwinner a10
antminer s1
antminer s2
antminer s3
antminer s3+
antminer s4
antminer s5
antminer s5+
antminer s7
antminer s9
antminer u1
antminer u2+
antminer u3
arm1136ej-s
arm1176jz-s
arm926ej-s
asicminer be blade
asicminer be cube
asicminer be prisma
asicminer be sapphire
asicminer be tube
athlon 64 3500+
athlon 64 x2 3800+
athlon 64 x2 4000+
athlon 64 x2 4400+
athlon 

In [11]:
dict_string = "mapping = {\n"

for file in os.listdir('mappings'):
    content = open(f"mappings/{file}").read()
    lines = content.split("\n")
    relevant_lines = lines[2:-2]
    for line in relevant_lines:
        # if a line has no comma at the end, add it
        part_before_comment = line.split("#")[0].strip()
        if not part_before_comment.endswith(","):
            if line.__contains__("#"):
                line = line.split("#")[0] + ", #" + line.split("#")[1]
            else:
                line += ","
        dict_string += line + "\n"

dict_string += "}"
code = f"""
{dict_string}
def map_hardware_to_table(string):
    return mapping.get(string, "not found")
""".strip()

# save to file
with open("hardware_mapping.py", "w") as file:
    file.write(code)

In [9]:
from hardware_mapping import map_hardware_to_table
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))

IndentationError: unexpected indent (hardware_mapping.py, line 1543)

In [ ]:
hardware_instances.head(10)

,date,hardware,hardware_mapped
0,2010-11-29 17:21:55,64-bit atom system,unknown
1,2011-01-21 18:36:10,gtx 460,not found
2,2011-01-21 18:36:10,gt330m,not found
3,2011-01-21 18:36:10,8800gtx,8800gtx
4,2011-01-21 18:36:10,macbook pro,not found
5,2011-02-19 07:33:30,6 gb of ram,unknown
6,2011-02-19 07:33:30,2x nvidia 1gb gtx 460s,gtx460
7,2011-02-19 07:33:30,core i7-950,not found
8,2011-02-27 21:32:08,n900,not found
9,2011-02-27 21:32:08,5870,5870


In [ ]:
efficiency = pd.read_csv('../../hardwarelist/hardware_merged.csv') # columns: hardware_name,Mhash/J
efficiency = efficiency.rename(columns={"hardware_name":"hardware_mapped"})

efficiency.head(2)

,hardware_mapped,Mhash/J
0,2x opteron 6128,32.4
1,2x opteron 6172,55.0


In [ ]:
joined = hardware_instances.merge(efficiency, on="hardware_mapped", how="left")
joined = joined.dropna(subset=["Mhash/J"])
joined.head(4)

,date,hardware,hardware_mapped,Mhash/J
6,2011-02-19 07:33:30,2x nvidia 1gb gtx 460s,gtx460,0.4270
9,2011-02-27 21:32:08,5870,5870,1.9060
25,2011-03-01 13:33:58,6870,6870,1.7264
26,2011-03-01 13:33:58,5870,5870,1.9060


In [ ]:
print(len(joined))

1708


In [ ]:
table = joined[["date","hardware_mapped","Mhash/J"]]
table = table.rename(columns={"hardware_mapped":"hardware_name"})
table["Mhash/J"] = table["Mhash/J"].astype(float).map(lambda x: x/1000000).map(lambda x: f"{x:.10f}")
table = table.rename(columns={"Mhash/J":"TH/J"})
table = table.sort_values("date")
table = table.reset_index(drop=True)
table.head(8)

,date,hardware_name,TH/J
0,2011-02-19 07:33:30,gtx460,0.0000004270
1,2011-02-27 21:32:08,5870,0.0000019060
2,2011-03-01 13:33:58,6870,0.0000017264
3,2011-03-01 13:33:58,5870,0.0000019060
4,2011-03-01 17:03:07,9400gt,0.0000000670
5,2011-03-02 03:06:15,4850,0.0000007956
6,2011-03-02 03:06:15,4850,0.0000007956
7,2011-03-03 12:24:02,6950,0.0000019399


In [ ]:
table.to_csv("hardware_instances_with_efficiency.csv", index=False)